In [26]:
import jdc

ModuleNotFoundError: No module named 'jdc'

In [1]:
class gramatica(object):
    def __init__(self):
        self.terminais = []
        self.variaveis = []
        self.inicial = []
        self.regras = []
        self.bonito = []

In [2]:
blabla = gramatica()

In [21]:
class gramatica(gramatica):
    def formataProd(self):
        self.regras =  ",".join(self.regras)
        self.regras = self.regras.replace(" ", "", 1)
        self.regras = self.regras.replace("   "," ")
        self.regras = self.regras.replace("  >  "," ")
        self.regras = self.regras.replace(" ,",",")
        self.regras = self.regras.split(", ")
        for i in range(len(self.regras)):
            self.regras[i] = self.regras[i].split(' ')
        self.regras[-1].remove('')

        
    def leGramatica(self, nomeArquivo):
        with open(nomeArquivo,"r") as arq:
            conteudo= arq.read().splitlines()
        for i in range(0,conteudo.index("#Regras")):
            conteudo[i] = conteudo[i].replace('\t', '')
            conteudo[i] = conteudo[i].replace('[', '')
            conteudo[i] = conteudo[i].replace(']', '')
            conteudo[i] = conteudo[i].replace(' ', '')
        for i in range(conteudo.index("#Regras"),len(conteudo)):
            conteudo[i] = conteudo[i].replace('\t', '')
            conteudo[i] = conteudo[i].replace('[', '')
            conteudo[i] = conteudo[i].replace(']', '')
        for i in range(len(conteudo)):
            if conteudo[i] == "#Terminais":
                for n in range(i+1,conteudo.index("#Variaveis")):
                    self.terminais.append(conteudo[n])
                    i+=1
            elif conteudo[i] == "#Variaveis":
                for n in range(i+1,conteudo.index("#Inicial")):
                    self.variaveis.append(conteudo[n])
                    i+=1
            elif conteudo[i] == "#Inicial":
                self.inicial = conteudo[i+1]
            elif conteudo[i] == "#Regras":
                for n in range(i+1,len(conteudo)):
                    self.regras.append(conteudo[n])
                    self.bonito.append(conteudo[n])
                    i+=1
        self.formataProd()

blabla = gramatica()
blabla.leGramatica("gramatica_exemplo3.txt")

In [19]:
blabla.leGramatica("gramatica_exemplo3.txt")

TypeError: sequence item 0: expected str instance, list found

In [22]:
class gramatica(gramatica):
    def defFormal(self):
        var = ", ".join(self.variaveis)
        term = ", ".join(self.terminais)
        prod = ",\n".join(self.bonito)
        print("G = ({%s},{%s},P ,%s)" % (var, term, self.inicial))
        print("P = {%s}" % (prod.replace(", ", ",\n")))

In [23]:
blabla.defFormal()

G = ({S, Z, B, X, Y, A},{a, b, u, v},P ,S)
P = { S  >  X   Y   Z ,
 A  >  a ,
 B  >  b ,
 X  >  A   X   A ,
 X  >  B   X   B ,
 X  >  Z ,
 X  >  V ,
 Y  >  A   Y   B ,
 Y  >  B   Y   A ,
 Y  >  Z ,
 Y  >  V ,
 Z  >  Z   u ,
 Z  >  Z   v ,
 Z  >  V ,
 Y  >  A ,
 A  >  X ,
 B  >  A ,
 J  >  K   a }


In [24]:
class gramatica(gramatica):
    def removeVazios(self):
        prodVazio = []
        #faz fecho com produções vazias diretas
        for prod in self.regras:
            for var in prod:
                if var == 'V':
                    prodVazio.append(prod[0])
        diretoVazio = prodVazio.copy()
        #faz fecho com produções vazias indiretas
        tam = 0
        while (len(prodVazio) != tam):
            tam = len(prodVazio)
            for i in range(len(self.regras)):
                naoGeraVazio = 0
                for j in range(1, len(self.regras[i])):
                    if self.regras[i][j] not in prodVazio:
                        naoGeraVazio = 1
                if not naoGeraVazio and self.regras[i][0] not in prodVazio:
                        prodVazio.append(self.regras[i][0])
        #remove produções vazias diretas
        for prod in self.regras:
            naoGeraVazio = 0
            if prod[1] == 'V':
                self.regras.remove(prod)
        #gera produções substituindo as ocorrências das variáveis do fecho
        for var in prodVazio:
            for prod in self.regras:
                for item in prod:
                    if var in  item:
                        novaProd = [item for item in prod if item == prod[0] or item != var]
                        if novaProd not in self.regras and len(novaProd) > 1:
                            self.regras.append(novaProd)
        """
        #gera produção vazia a partir do inicial caso necessário
        if self.inicial in prodVazio:
            self.regras.append([self.inicial, 'V'])
        """

In [25]:
blabla = gramatica()
blabla.leGramatica("gramatica_exemplo3.txt")
blabla.removeVazios()
print(self.regras)

NameError: name 'self' is not defined

In [ ]:
class gramatica(gramatica):
    def removeUnit(self):
        #cria fechos
        fechos = []
        for var in self.variaveis:
            fechos.append([var])
        #aumenta fecho com produções unitárias correspondentes
        for i in range(len(fechos)):
            tam = 0
            j = 0
            while tam != len(fechos[i]):
                tam = len(fechos[i])
                for prod in self.regras:
                    if prod[0] == fechos[i][j] and len(prod) == 2 and prod[1] not in fechos[i] and prod[1] in self.variaveis:
                        fechos[i].append(prod[1])
                j+=1        
        #remove produções unitárias
        i = 0
        while i < len(self.regras):
            if len(self.regras[i]) == 2 and self.regras[i][1] in self.variaveis:
                self.regras.remove(self.regras[i])
            else:
                i+=1
        #guacumula produções
        for fecho in fechos:
            for i in range(1,len(fecho)):
                for prod in self.regras:
                    if fecho[i] == prod[0]:
                        novaProd = []
                        novaProd.append(fecho[0])
                        for v in prod[1:]:
                            novaProd.append(v)
                        if novaProd not in self.regras:
                            self.regras.append(novaProd)

In [ ]:
blabla.removeUnit()
print(self.regras)

In [ ]:
def removeInuteis(self):
        uteis = []
        #inicializa com variáveis que chegam em terminais
        for prod in self.regras:
            for item in prod[1:]:
                if item in self.terminais and prod[0] not in uteis:
                    uteis.append(prod[0])
        #elimina casos com variáveis do lado direito
        bla =[]
        #print(uteis)
        for var in uteis:
            for prod in self.regras:
                if prod[0] == var:
                    flag = 0
                    for l in prod[1:]:
                        if l in self.variaveis:
                            flag = 1
                    if flag == 0 and prod[0] not in uteis:
                        bla.append(prod[0])

        print(bla)
        input("k")

        """
        criar lista com todas variaveis que levam APENAS a terminais (nao pode haver variavel do lado direito da produção)        
        """

        #gera lista de símbolos antingíveis
        atingiveis = [self.inicial]
        for prod in self.regras:
            if prod[0] == self.inicial:
                for i in range(1,len(prod)):
                    if prod[i] not in self.terminais and prod[i] not in atingiveis:
                        atingiveis.append(prod[i])
        #exclui produções de variáveis que não levam a terminais
#        self.regras = filter(lambda x: x in uteis, self.regras)

In [ ]:
blabla.removeInuteis()
print(self.regras)